<a href="https://colab.research.google.com/github/Sineos/sineos.github.io/blob/main/pa_cube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Klipper Pressure Advance Cube using FullControl

press ctrl+F9 to run all cells in this notebook, or press shift+enter to run each cell sequentially 

if you change one of the code cells, make sure you run it and all subsequent cells again (in order)

*this document is a jupyter notebook - if they're new to you, check out how they work: [link](https://www.google.com/search?q=ipynb+tutorial), [link](https://jupyter.org/try-jupyter/retro/notebooks/?path=notebooks/Intro.ipynb), [link](https://colab.research.google.com/)*

### Description

the next code cell may take a while because running it causes several things to happen:
- connect to a google colab server -> download the fullcontrol code -> install the fullcontrol code
- generate the PA Cube calibration gcode
- generate a preview plot of the gcode
- download the gcode file

### Usage
Just print the resulting gcode file

### Warning
There are no sanity / safety checks in the code. Be sure to have correctly set bed dimension and cube dimensions.

In [1]:
!pip install git+https://github.com/FullControlXYZ/fullcontrol --quiet
import fullcontrol as fc
import math
from google.colab import files

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# printer/gcode parameters

# Bed Parameter [mm]
bed_size_x = 263
bed_size_y = 249

# Cube size [mm]
cube_size_x = 100
cube_size_y = 200
cube_size_height = 50

# Temperature Parameters
temp_nozzle = 210
temp_bed = 65

# Velocity Parameter [mm/s]
speed_print = 100
speed_travel = 150
speed_slow_factor = 0.2
speed_first_layer = 50

# Extrusion Parameters [mm]
extrusion_width = 0.42
extrusion_height = 0.2
filament_diameter = 1.75

# Pressure Advance Tuning Tower Parameters
pa_start = 0.2
pa_step_delta = 0.002
pa_step_height = 2

# Cooling Parameter
fan_percent = 0

# Miscellaneous Parameters
printer_name = 'custom'
design_name = 'pa_cube'
e_units = "mm"


In [8]:
# convert velocity from mm/s to mm/min
speed_print_min = speed_print * 60
speed_travel_min = speed_travel * 60
speed_first_layer_min = speed_first_layer * 60

# create the initialize procedure (i.e. start_steps)
initial_settings = {
    "extrusion_width": extrusion_width,
    "extrusion_height": extrusion_height,
    "e_units": e_units,
    "dia_feed": filament_diameter,
    "print_speed": speed_print_min,
    "travel_speed": speed_travel_min,
    "primer": "no_primer"
}
gcode_controls = fc.GcodeControls(printer_name=printer_name, initialization_data=initial_settings)

# Start Gcode
start_steps = []
start_steps.append(fc.ManualGcode(text='\n;##### beginning of start gcode'))
start_steps.append(fc.ManualGcode(text='G28 ; home'))
start_steps.append(fc.GcodeComment(text='heat bed 10 degrees too hot'))
start_steps.append(fc.Buildplate(temp=temp_bed + 10, wait=True))
start_steps.append(fc.GcodeComment(text='allow bed to cool to the correct temp and heat up nozzle'))
start_steps.append(fc.Hotend(temp=temp_nozzle, wait=False))
start_steps.append(fc.Buildplate(temp=temp_bed, wait=True))
start_steps.append(fc.Hotend(temp=temp_nozzle, wait=True))
start_steps.append(fc.Fan(speed_percent=fan_percent))
start_steps.append(fc.Extruder(relative_gcode=False))

# Prime Line
prime_steps = []
prime_steps.append(fc.GcodeComment(text='##### Prime'))
prime_steps.extend(fc.travel_to(fc.Point(x=0.5, y=10, z=0.3, color=[0,0,1])))
prime_steps.append(fc.Extruder(on=True))
prime_steps.append(fc.Point(y=bed_size_y-10, color=[0,0,1]))
prime_steps.append(fc.Point(x=0.5 + extrusion_width))
prime_steps.append(fc.Point(y=10))
prime_steps.append(fc.Extruder(relative_gcode=False))

# Start the design
layers = math.ceil(cube_size_height / extrusion_height)
initial_z = extrusion_height
mid_bed_x = bed_size_x / 2.
mid_bed_y = bed_size_y / 2.
brim_steps = []

# Initialize the tuning tower
brim_steps.append(fc.ManualGcode(text='TUNING_TOWER COMMAND=SET_PRESSURE_ADVANCE PARAMETER=ADVANCE START=%s STEP_DELTA=%s STEP_HEIGHT=%s'%(pa_start, pa_step_delta, pa_step_height)))

# Create brim
brim_start_x = (mid_bed_x - (cube_size_x / 2.)) + (-5 * extrusion_width)
brim_start_y = (mid_bed_y - (cube_size_y / 2.)) + (-5 * extrusion_width)
brim_steps.append(fc.GcodeComment(text='##### Brim'))
brim_steps.extend(fc.travel_to(fc.Point(x=brim_start_x, y=brim_start_y)))
brim_steps.extend(fc.travel_to(fc.Point(z=extrusion_height)))
brim_steps.append(fc.Extruder(on=True))
brim_steps.append(fc.Printer(print_speed=speed_first_layer_min))
for brim in range(-5, 5):
  x_start = (mid_bed_x - (cube_size_x / 2.)) + (brim * extrusion_width)
  y_start = (mid_bed_y - (cube_size_y / 2.)) + (brim * extrusion_width)
  x_length = cube_size_x - (2 * brim * extrusion_width)
  y_length = cube_size_y - (2 * brim * extrusion_width)
  rectangle_steps = fc.rectangleXY(fc.Point(x=x_start, y=y_start, z=initial_z), x_length, y_length)
  brim_steps.extend(rectangle_steps)

# Travel to PA Cube start
x_start = mid_bed_x
y_start = (mid_bed_y - (cube_size_y / 2.))
brim_steps.append(fc.GcodeComment(text='##### Move to Cube Start'))
brim_steps.extend(fc.travel_to(fc.Point(z=initial_z + extrusion_height)))
brim_steps.extend(fc.travel_to(fc.Point(x=x_start, y=y_start)))

# Create PA cube
cube_steps_prepare = []
cube_steps = []
cube_steps_prepare.append(fc.GcodeComment(text='##### Cube Wall 1'))
cube_steps_prepare.extend(fc.travel_to(fc.Point(z=initial_z + extrusion_height)))
cube_steps_prepare.extend(fc.travel_to(fc.Point(y=mid_bed_y - (cube_size_y / 2.))))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min * speed_slow_factor))
cube_steps_prepare.append(fc.Point(x=mid_bed_x + (cube_size_x / 4.), color=[0,1,0]))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min))
cube_steps_prepare.append(fc.Point(x=mid_bed_x + (cube_size_x / 2.), color=[1,0,0]))
cube_steps_prepare.append(fc.Point(y=mid_bed_y, color=[1,0,0]))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min * speed_slow_factor))
cube_steps_prepare.append(fc.Point(y=mid_bed_y + (cube_size_y / 2.), color=[0,1,0]))
cube_steps_prepare.append(fc.Point(x=mid_bed_x + (cube_size_x / 4.), color=[0,1,0]))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min))
cube_steps_prepare.append(fc.Point(x=mid_bed_x - (cube_size_x / 4.), color=[1,0,0]))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min * speed_slow_factor))
cube_steps_prepare.append(fc.Point(x=mid_bed_x - (cube_size_x / 2.), color=[0,1,0]))
cube_steps_prepare.append(fc.Point(y=mid_bed_y, color=[0,1,0]))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min))
cube_steps_prepare.append(fc.Point(y=mid_bed_y - (cube_size_y / 2.), color=[1,0,0]))
cube_steps_prepare.append(fc.Point(x=mid_bed_x - (cube_size_x / 4.), color=[1,0,0]))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min * speed_slow_factor))
cube_steps_prepare.append(fc.Point(x=mid_bed_x, color=[0,1,0]))
cube_steps_prepare.append(fc.GcodeComment(text='##### Cube Wall 2'))
cube_steps_prepare.extend(fc.travel_to(fc.Point(y=mid_bed_y - (cube_size_y / 2.) - extrusion_width)))
cube_steps_prepare.extend(fc.travel_to(fc.Point(z=initial_z + extrusion_height)))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min * speed_slow_factor))
cube_steps_prepare.append(fc.Point(x=mid_bed_x + (cube_size_x / 4.), color=[0,1,0]))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min))
cube_steps_prepare.append(fc.Point(x=mid_bed_x + (cube_size_x / 2.) + extrusion_width, color=[1,0,0]))
cube_steps_prepare.append(fc.Point(y=mid_bed_y, color=[1,0,0]))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min * speed_slow_factor))
cube_steps_prepare.append(fc.Point(y=mid_bed_y + (cube_size_y / 2.) + extrusion_width, color=[0,1,0]))
cube_steps_prepare.append(fc.Point(x=mid_bed_x + (cube_size_x / 4.), color=[0,1,0]))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min))
cube_steps_prepare.append(fc.Point(x=mid_bed_x - (cube_size_x / 4.), color=[1,0,0]))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min * speed_slow_factor))
cube_steps_prepare.append(fc.Point(x=mid_bed_x - (cube_size_x / 2.) - extrusion_width, color=[0,1,0]))
cube_steps_prepare.append(fc.Point(y=mid_bed_y, color=[0,1,0]))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min))
cube_steps_prepare.append(fc.Point(y=mid_bed_y - (cube_size_y / 2.) - extrusion_width, color=[1,0,0]))
cube_steps_prepare.append(fc.Point(x=mid_bed_x - (cube_size_x / 4.), color=[1,0,0]))
cube_steps_prepare.append(fc.Printer(print_speed=speed_print_min * speed_slow_factor))
cube_steps_prepare.append(fc.Point(x=mid_bed_x, color=[0,1,0]))
# Pull up cube_steps_prepare along the Z vector
cube_steps = fc.move(cube_steps_prepare, fc.Vector(z=extrusion_height), copy=True, copy_quantity=layers)

# Create end gcode
end_steps = []
end_steps.append(fc.GcodeComment(text='##### End Gcode'))
end_steps.append(fc.ManualGcode(text='M400; wait for buffer to clear'))
end_steps.append(fc.ManualGcode(text='G92 E0 ; zero the extruder'))
end_steps.append(fc.ManualGcode(text='G1 E-1.0' + str(speed_print_min) + ' ; retract filament'))
end_steps.append(fc.ManualGcode(text='TURN_OFF_HEATERS'))
end_steps.append(fc.ManualGcode(text='M107 ; turn off fan'))
end_steps.append(fc.ManualGcode(text='G91 ; relative positioning'))
end_steps.append(fc.ManualGcode(text='G1 Z15 F' + str(speed_travel_min) + ' ; move nozzle up 15mm'))
end_steps.append(fc.ManualGcode(text='BED_MESH_CLEAR ; clear any active mesh'))

# Combine start procedure and design to create the overall procedure
steps = start_steps + prime_steps + brim_steps + cube_steps + end_steps
print(fc.transform(steps, 'gcode', gcode_controls))

; Time to print!!!!!
; GCode created with FullControl - tell us what you're printing!
; info@fullcontrol.xyz or tag FullControlXYZ on Twitter/Instagram/LinkedIn/Reddit/TikTok

;##### beginning of start gcode
G28 ; home
; heat bed 10 degrees too hot
M190 S75 ; set bed temp and wait
; allow bed to cool to the correct temp and heat up nozzle
M104 S210 ; set hotend temp and continue
M190 S65 ; set bed temp and wait
M109 S210  ; set hotend temp and wait
M106 S0 ; set fan speed
M82 ; absolute extrusion
G92 E0 ; reset extrusion position to zero
; ##### Prime
G0 F9000 X0.5 Y10.0 Z0.3
G1 F6000 Y239.0 E7.9974
G1 X0.92 E8.01207
G1 Y10.0 E16.0095
M82 ; absolute extrusion
G92 E0 ; reset extrusion position to zero
TUNING_TOWER COMMAND=SET_PRESSURE_ADVANCE PARAMETER=ADVANCE START=0.2 STEP_DELTA=0.002 STEP_HEIGHT=2
; ##### Brim
G0 F9000 X79.4 Y22.4
G0 F9000 Z0.2
G1 F3000 X183.6 E3.63899
G1 Y226.6 E10.7703
G1 X79.4 E14.4093
G1 Y22.4 E21.5406
G1 X79.82 Y22.82 E21.5613
G1 X183.18 E25.171
G1 Y226.18 E32.2

In [7]:
# preview the design
fc.transform(steps, 'plot', fc.PlotControls(color_type='manual'))

In [45]:
# create and download gcode for the design
gcode = fc.transform(steps, 'gcode', gcode_controls)
open(f'{design_name}.gcode', 'w').write(gcode)
files.download(f'{design_name}.gcode')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>